In [1]:
# 本代码由可视化策略环境自动生成 2021年8月24日23:10
# 本代码单元只能在可视化模式下编辑。您也可以拷贝代码，粘贴到新建的代码单元或者策略，然后修改。


def calcu_relative_ret(df):
    start_date = min(m11.data.read_df().date).strftime('%Y-%m-%d')
    end_date = max(m11.data.read_df().date).strftime('%Y-%m-%d')
   
    sz50_df = D.history_data(
    '000016.SHA',
    start_date=(pd.to_datetime(start_date) - datetime.timedelta(days=10)).strftime('%Y-%m-%d'),  # 多取几天的数据
    end_date=end_date)[['date', 'close']].rename(columns={'close': 'sz50_close'})
    df = df[['date', 'close_0']].reset_index().merge(sz50_df, on='date', how='left').set_index('index')
    return df['close_0'].pct_change() - df['sz50_close'].pct_change()

def groupby_calcu_relative_ret(df, close_0):
    return df.groupby('instrument', group_keys=False).apply(calcu_relative_ret)

m12_user_functions_bigquant_run = {
    'groupby_calcu_relative_ret':  groupby_calcu_relative_ret
}


def calcu_relative_ret(df):
    start_date = min(m4.data.read_df().date).strftime('%Y-%m-%d')
    end_date = max(m4.data.read_df().date).strftime('%Y-%m-%d')
    hs800_df = D.history_data(
    '000016.SHA',
    start_date=(pd.to_datetime(start_date) - datetime.timedelta(days=10)).strftime('%Y-%m-%d'),  # 多取几天的数据
    end_date=end_date)[['date', 'close']].rename(columns={'close': 'hs800_close'})
    df = df[['date', 'close_0']].reset_index().merge(hs800_df, on='date', how='left').set_index('index')
    return df['close_0'].pct_change() - df['hs800_close'].pct_change()

def groupby_calcu_relative_ret(df, close_0):
    return df.groupby('instrument', group_keys=False).apply(calcu_relative_ret)

m2_user_functions_bigquant_run = {
    'groupby_calcu_relative_ret':  groupby_calcu_relative_ret
}


# 回测引擎：每日数据处理函数，每天执行一次
def m15_handle_data_bigquant_run(context, data):
    # 按日期过滤得到今日的预测数据
    ranker_prediction = context.ranker_prediction[
        context.ranker_prediction.date == data.current_dt.strftime('%Y-%m-%d')]

    # 1. 资金分配
    # 平均持仓时间是hold_days，每日都将买入股票，每日预期使用 1/hold_days 的资金
    # 实际操作中，会存在一定的买入误差，所以在前hold_days天，等量使用资金；之后，尽量使用剩余资金（这里设置最多用等量的1.5倍）
    is_staging = context.trading_day_index < context.options['hold_days'] # 是否在建仓期间（前 hold_days 天）
    cash_avg = context.portfolio.portfolio_value / context.options['hold_days']
    cash_for_buy = min(context.portfolio.cash, (1 if is_staging else 1.5) * cash_avg)
    cash_for_sell = cash_avg - (context.portfolio.cash - cash_for_buy)
    positions = {e.symbol: p.amount * p.last_sale_price
                 for e, p in context.perf_tracker.position_tracker.positions.items()}

    # 2. 生成卖出订单：hold_days天之后才开始卖出；对持仓的股票，按机器学习算法预测的排序末位淘汰
    if not is_staging and cash_for_sell > 0:
        equities = {e.symbol: e for e, p in context.perf_tracker.position_tracker.positions.items()}
        instruments = list(reversed(list(ranker_prediction.instrument[ranker_prediction.instrument.apply(
                lambda x: x in equities and not context.has_unfinished_sell_order(equities[x]))])))
        # print('rank order for sell %s' % instruments)
        for instrument in instruments:
            context.order_target(context.symbol(instrument), 0)
            cash_for_sell -= positions[instrument]
            if cash_for_sell <= 0:
                break

    # 3. 生成买入订单：按机器学习算法预测的排序，买入前面的stock_count只股票
    buy_cash_weights = context.stock_weights
    buy_instruments = list(ranker_prediction.instrument[:len(buy_cash_weights)])
    max_cash_per_instrument = context.portfolio.portfolio_value * context.max_cash_per_instrument
    for i, instrument in enumerate(buy_instruments):
        cash = cash_for_buy * buy_cash_weights[i]
        if cash > max_cash_per_instrument - positions.get(instrument, 0):
            # 确保股票持仓量不会超过每次股票最大的占用资金量
            cash = max_cash_per_instrument - positions.get(instrument, 0)
        if cash > 0:
            context.order_value(context.symbol(instrument), cash)

# 回测引擎：准备数据，只执行一次
def m15_prepare_bigquant_run(context):
    pass

# 回测引擎：初始化函数，只执行一次
def m15_initialize_bigquant_run(context):
    # 加载预测数据
    context.ranker_prediction = context.options['data'].read_df()

    # 系统已经设置了默认的交易手续费和滑点，要修改手续费可使用如下函数
    context.set_commission(PerOrder(buy_cost=0.0003, sell_cost=0.0013, min_cost=5))
    # 预测数据，通过options传入进来，使用 read_df 函数，加载到内存 (DataFrame)
    # 设置买入的股票数量，这里买入预测股票列表排名靠前的5只
    stock_count = 5
    # 每只的股票的权重，如下的权重分配会使得靠前的股票分配多一点的资金，[0.339160, 0.213986, 0.169580, ..]
    context.stock_weights = T.norm([1 / math.log(i + 2) for i in range(0, stock_count)])
    # 设置每只股票占用的最大资金比例
    context.max_cash_per_instrument = 0.2
    context.options['hold_days'] = 5


m1 = M.input_features.v1(
    features="""return_5
return_10
return_20
volume_0/mean(close_0,3)
avg_amount_0/avg_amount_5  
avg_amount_5/avg_amount_10  
sum(mf_net_amount_xl_0, 5) 
sum(mf_net_amount_l_0, 5)
close_0
market_cap_float_0
groupby_calcu_relative_ret(close_0)

 
"""
)

m10 = M.instruments.v2(
    start_date='2016-01-01',
    end_date='2018-01-22',
    market='CN_STOCK_A',
    instrument_list="""600000.SHA
600010.SHA
600015.SHA
600016.SHA
600018.SHA
600028.SHA
600029.SHA
600030.SHA
600036.SHA
600048.SHA
600050.SHA
600089.SHA
600100.SHA
600104.SHA
600109.SHA
600111.SHA
600150.SHA
600196.SHA
600256.SHA
600332.SHA
600340.SHA
600372.SHA
600406.SHA
600485.SHA
600518.SHA
600519.SHA
600547.SHA
600583.SHA
600585.SHA
600606.SHA
600637.SHA
600690.SHA
600703.SHA
600795.SHA
600832.SHA
600837.SHA
600887.SHA
600893.SHA
600919.SHA
600958.SHA
600999.SHA
601006.SHA
601088.SHA
601118.SHA
601166.SHA
601169.SHA
601186.SHA
601198.SHA
601211.SHA
601229.SHA
601288.SHA
601299.SHA
601318.SHA
601328.SHA
601336.SHA
601377.SHA
601390.SHA
601398.SHA
601601.SHA
601628.SHA
601668.SHA
601669.SHA
601688.SHA
601727.SHA
601766.SHA
601788.SHA
601800.SHA
601818.SHA
601857.SHA
601881.SHA
601901.SHA
601919.SHA
601985.SHA
601988.SHA
601989.SHA
601998.SHA""",
    max_count=0
)

m11 = M.general_feature_extractor.v6(
    instruments=m10.data,
    features=m1.data,
    start_date='',
    end_date='',
    before_start_days=0
)

m12 = M.derived_feature_extractor.v2(
    input_data=m11.data,
    features=m1.data,
    date_col='date',
    instrument_col='instrument',
    user_functions=m12_user_functions_bigquant_run,
    m_cached=False
)

m13 = M.dropnan.v1(
    input_data=m12.data
)

m18 = M.instruments.v2(
    start_date='2011-01-01',
    end_date='2016-01-01',
    market='CN_STOCK_A',
    instrument_list="""600000.SHA
600010.SHA
600015.SHA
600016.SHA
600018.SHA
600028.SHA
600029.SHA
600030.SHA
600036.SHA
600048.SHA
600050.SHA
600089.SHA
600100.SHA
600104.SHA
600109.SHA
600111.SHA
600150.SHA
600196.SHA
600256.SHA
600332.SHA
600340.SHA
600372.SHA
600406.SHA
600485.SHA
600518.SHA
600519.SHA
600547.SHA
600583.SHA
600585.SHA
600606.SHA
600637.SHA
600690.SHA
600703.SHA
600795.SHA
600832.SHA
600837.SHA
600887.SHA
600893.SHA
600919.SHA
600958.SHA
600999.SHA
601006.SHA
601088.SHA
601118.SHA
601166.SHA
601169.SHA
601186.SHA
601198.SHA
601211.SHA
601229.SHA
601288.SHA
601299.SHA
601318.SHA
601328.SHA
601336.SHA
601377.SHA
601390.SHA
601398.SHA
601601.SHA
601628.SHA
601668.SHA
601669.SHA
601688.SHA
601727.SHA
601766.SHA
601788.SHA
601800.SHA
601818.SHA
601857.SHA
601881.SHA
601901.SHA
601919.SHA
601985.SHA
601988.SHA
601989.SHA
601998.SHA""",
    max_count=0
)

m4 = M.general_feature_extractor.v6(
    instruments=m18.data,
    features=m1.data,
    start_date='',
    end_date='',
    m_cached=False
)

m2 = M.derived_feature_extractor.v2(
    input_data=m4.data,
    features=m1.data,
    date_col='date',
    instrument_col='instrument',
    user_functions=m2_user_functions_bigquant_run
)

m5 = M.advanced_auto_labeler.v2(
    instruments=m18.data,
    label_expr="""# #号开始的表示注释
# 0. 每行一个，顺序执行，从第二个开始，可以使用label字段
# 1. 可用数据字段见 https://bigquant.com/docs/data_history_data.html
#   添加benchmark_前缀，可使用对应的benchmark数据
# 2. 可用操作符和函数见 `表达式引擎 <https://bigquant.com/docs/big_expr.html>`_

# 计算收益：5日收盘价(作为卖出价格)除以明日开盘价(作为买入价格)
shift(close, -5) / shift(open, -1)

# 极值处理：用1%和99%分位的值做clip
clip(label, all_quantile(label, 0.01), all_quantile(label, 0.99))

# 将分数映射到分类，这里使用20个分类
all_wbins(label, 20)

# 过滤掉一字涨停的情况 (设置label为NaN，在后续处理和训练中会忽略NaN的label)
where(shift(high, -1) == shift(low, -1), NaN, label)
""",
    start_date='',
    end_date='',
    benchmark='000300.SHA',
    drop_na_label=True,
    cast_label_int=True,
    user_functions={}
)

m6 = M.join.v3(
    data1=m5.data,
    data2=m2.data,
    on='date,instrument',
    how='inner',
    sort=False
)

m7 = M.dropnan.v1(
    input_data=m6.data
)

m8 = M.stock_ranker_train.v5(
    training_ds=m7.data,
    features=m1.data,
    learning_algorithm='排序',
    number_of_leaves=30,
    minimum_docs_per_leaf=1000,
    number_of_trees=20,
    learning_rate=0.1,
    max_bins=1023,
    feature_fraction=1,
    m_lazy_run=False
)

m14 = M.stock_ranker_predict.v5(
    model=m8.model,
    data=m13.data,
    m_lazy_run=False
)

m15 = M.trade.v3(
    instruments=m10.data,
    options_data=m14.predictions,
    start_date='',
    end_date='',
    handle_data=m15_handle_data_bigquant_run,
    prepare=m15_prepare_bigquant_run,
    initialize=m15_initialize_bigquant_run,
    volume_limit=0.025,
    order_price_field_buy='open',
    order_price_field_sell='close',
    capital_base=1000000,
    benchmark='000300.SHA',
    auto_cancel_non_tradable_orders=True,
    data_frequency='daily',
    price_type='后复权',
    plot_charts=True,
    backtest_only=False
)


[2018-03-30 21:08:45.163243] INFO: bigquant: input_features.v1 开始运行..
[2018-03-30 21:08:45.174400] INFO: bigquant: 命中缓存
[2018-03-30 21:08:45.178868] INFO: bigquant: input_features.v1 运行完成[0.015629s].
[2018-03-30 21:08:45.206212] INFO: bigquant: instruments.v2 开始运行..
[2018-03-30 21:08:45.214191] INFO: bigquant: 命中缓存
[2018-03-30 21:08:45.221861] INFO: bigquant: instruments.v2 运行完成[0.015638s].
[2018-03-30 21:08:45.268631] INFO: bigquant: general_feature_extractor.v6 开始运行..
[2018-03-30 21:08:45.291510] INFO: bigquant: 命中缓存
[2018-03-30 21:08:45.298167] INFO: bigquant: general_feature_extractor.v6 运行完成[0.029525s].
[2018-03-30 21:08:45.443384] INFO: bigquant: derived_feature_extractor.v2 开始运行..
[2018-03-30 21:08:45.623471] INFO: derived_feature_extractor: 提取完成 avg_amount_0/avg_amount_5, 0.003s
[2018-03-30 21:08:45.649524] INFO: derived_feature_extractor: 提取完成 avg_amount_5/avg_amount_10, 0.020s
[2018-03-30 21:09:57.022507] INFO: derived_feature_extractor: 提取完成 groupby_calcu_relative_ret(close_

[2018-03-30 21:13:50.964857] INFO: bigquant: backtest.v7 运行完成[28.605077s].
